In [1]:
import numpy.random as npr
import numpy as np

import torch 
from jak.transformer import XFormer

import pandas as pd

import matplotlib.pyplot as plt
my_cmap = plt.get_cmap("plasma")


import numpy as np

In [2]:
df = pd.read_csv("../data/train_JAK.csv")
df.head()

,SMILES,measurement_type,measurement_value,Kinase_name
0,C#CCCOC(=O)N1CCC(n2cc(C(N)=O)c(Nc3ccc(F)cc3)n2...,pIC50,6.81,JAK2
1,C#CCCOC(=O)N1CCC(n2cc(C(N)=O)c(Nc3ccc(F)cc3)n2...,pIC50,8.05,JAK1
2,C#CCN(c1ccc(C#N)cn1)C1CCN(c2ncnc3[nH]ccc23)C1,pIC50,10.26,JAK2
3,C#CCN(c1ccc(C#N)cn1)C1CCN(c2ncnc3[nH]ccc23)C1,pIC50,10.26,JAK1
4,C#CCNCC1CCC(c2nnn3cnc4[nH]ccc4c23)CC1,pIC50,7.36,JAK2


In [5]:
smiles_vocab = "#()+-1234567=BCFHINOPS[]cilnors"
model = XFormer(vocab=smiles_vocab, token_dim=33, seq_length=100, lr=1e-3, device="cpu", tag="inference")
model_state_dict = torch.load("../parameters/xformer_x003_seed42/tag_xformer_x003_seed42_epoch299.pt", \
                              map_location=model.my_device)
model.load_state_dict(model_state_dict)

<All keys matched successfully>

In [6]:
# encode SMILES into numpy arrays of features

num_values = len(df["Kinase_name"].unique()) * len(df["measurement_type"].unique())

x = torch.Tensor()
y = torch.Tensor()

x_dict = {}
y_dict = {}

m_dict = {"pIC50": 0 , "pKi": 1}
e_dict = {"JAK1": 0, "JAK2": 1, "JAK3": 2, "TYK2": 3}

number_enzymes = 4
with torch.no_grad():
    for ii in range(len(df)):
        my_smile = df["SMILES"][ii]
        if my_smile not in x_dict.keys():
            encoded = model.encode(my_smile).reshape(1, -1) #unsqueeze(0)
            x_dict[my_smile] = encoded
            y_dict[my_smile] = [0.0] * num_values

        index = m_dict[df["measurement_type"][ii]] * number_enzymes + e_dict[df["Kinase_name"][ii]]
        y_dict[my_smile][index] = df["measurement_value"][ii]


In [7]:
x_dict[my_smile].shape

torch.Size([1, 3300])

In [8]:

x = torch.Tensor()
y = torch.Tensor()

for key in x_dict.keys():
    
    x = torch.cat([x, x_dict[key]])
    y = torch.cat([y, torch.tensor(y_dict[key]).reshape(1, num_values)])
    
print(x.shape, y.shape)    

torch.Size([4084, 3300]) torch.Size([4084, 8])


In [9]:
x.shape, y.shape

torch.save(x, "../data/kinase_x003.pt")
torch.save(y, "../data/kinase_y003.pt")

np.save("../data/kinase_x003.npy", x.numpy())
np.save("../data/kinase_y003.npy", y.numpy())